# 実際のデータを読み込む
## 佐賀県の人口を例に

In [1]:
import pandas
import re

In [2]:
#url = 'https://data.bodik.jp/dataset/77e0cc66-c15d-4473-b3df-2664fe8e2e63/resource/8dc71515-526a-4168-866c-05d2cc8dad7b/download/jinkou.xlsx'
url = 'jinkou.xlsx'

## 表中の和暦を西暦に変換する辞書型
表中では、年が和暦であり、さらに元号が省略されている部分がある。
さらに、「大正9年」と「昭和5年」の間が10年で、他の間が5年である。
そこで、表中の年を西暦(整数)へ変更する辞書型を準備しておく。

In [3]:
# wareki = {
#         '大正9年':1920, '昭和5年':1930, '10年':1935, '15年':1940,
#         '20年':1945, '25年':1950, '30年':1955, '35年':1960,'40年':1965,
#         '45年':1970, '50年':1975, '55年':1980, '60年':1985, '平成2年':1990,
#         '7年':1995, '12年':2000, '17年':2005, '22年':2010, '27年':2015}
def convertWareki(columns:pandas.Index) -> dict[str,int]:
    y = 1930
    wareki:dict[str,int] = dict()
    for k in columns:
        if k == '大正9年':
            wareki[k]=1920
        else:
            wareki[k]=y
            y+=5
    return wareki

# 行名の置き換え
- A列とB列に行名があるが、二列に分かれている
- V列は行名として使えそう
- V列の行名から、A列とB列を用いて、より適切な名前を辞書型で生成

In [4]:
def getIndex(url:str) -> dict[str,str]:
    """
    URLで指定されたExcelで、V列に対応する適切な行名を取得する
    """
    with pandas.ExcelFile(url) as f:
        dd:pandas.DataFrame  = pandas.read_excel(f, header = 3,  usecols = 'A:B,V',
        index_col = 2, skiprows = [4, 18, 27], skipfooter = 8) 
    newIndex :dict[str,str]= dict()
    for k in dd.index:
        v:str = str(dd['項'][k])
        if type(v) != str:
            v = str(dd['目'][k])
        v = re.sub(r'\s','',v)
        newIndex[k] = v.strip()
    return newIndex

## Excelファイル読み込み
- C列からV列を読み込む
- 一部の行を無視
- フッタ部分を無視
- 分かりやすい行名に入れ替え

In [5]:
def readData(url:str) -> pandas.DataFrame:
    """
    urlで指定されたExcelファイルを読む
    """
    with pandas.ExcelFile(url) as f:
        data:pandas.DataFrame = pandas.read_excel(f, header = 3, usecols = 'C:V',
             index_col = 19, skiprows = [4, 18, 27], skipfooter = 8) 
    keys: dict[str, str] = getIndex(url)
    wareki: dict[str, int] = convertWareki(data.columns)
    return data.rename(columns = wareki, index = keys)

## メイン部分

- CSVファイルを読む。
- ラベルの整理を行う。
- 整理したラベルを印刷する。

In [6]:
df :pandas.DataFrame = readData(url)
print(df)
newIndex:dict[str,str] = getIndex(url)
print('新しく取得した行のラベル')
for k in newIndex.keys():
    print(f'{k}->{newIndex[k]}')


                    1920      1930      1935      1940      1945      1950  \
項　　目                                                                         
全国              55963053  64450005  69254148  73114308  71998104  84114574   
佐賀県               673895    691565    686117    701517    830431    945082   
nan                33528     46183     81212     81748     97354    118627   
nan               640367    645382    604905    619769    733077    826455   
増加数                    －     17670     -5448     15400    128914    114651   
増加率(%)                 －       2.6      -0.8       2.2      18.4      13.8   
全国人口に対する割合(%)        1.2      1.07      0.99      0.96      1.15      1.12   
面積               2443.59    2443.9   2449.03   2449.03   2449.03   2403.74   
人口密度(1k㎡あたり）       275.8       283     280.2     286.4     339.1     393.2   
市町村数                 135       132       125       125       122       122   
nan                    1         1         2         2         2